In [1]:
!pip install tabula.py

from google.colab import files
uploaded=files.upload()
import pandas as pd
import numpy as np
eps=np.finfo(float).eps
import pprint
from numpy import log2 as log

Data=pd.read_csv("Purchase_new.csv")
print(Data)

Data.shape
x = Data.drop('Purchase',axis=1)
y = Data['Purchase']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 58.8 MB/s eta 0:00:00


Saving Purchase_new.csv to Purchase_new.csv
   Holiday Discount Free Delivery Purchase
0       No      Yes           Yes      Yes
1       No      Yes           Yes      Yes
2       No       No            No       No
3      yes      Yes           Yes      Yes
4      yes      Yes           Yes      Yes
5      yes       No            No       No
6      yes      Yes            No      Yes
7       No      Yes           Yes      Yes
8      yes      Yes           Yes      Yes
9      yes      Yes           Yes      Yes
10     yes       No           Yes      Yes
11     yes       No            No       No
12     yes      Yes           Yes      Yes
13     yes      Yes           Yes      Yes
14     yes      Yes           Yes      Yes
15      No      Yes           Yes      Yes
16     yes       No           Yes      Yes
17      No      Yes            No      Yes
18     yes       No            No      Yes
19     yes       No           Yes      Yes
20      No      Yes           Yes      Yes
21     yes

In [2]:
def find_entropy(df):
  Class=df.keys()[-1]
  entropy=0
  values=df[Class].unique()
  for value in values:
    a=df[Class].value_counts()[value]/len(df[Class])
    entropy+=-a*np.log2(a)
  return entropy

def conditional_entropy(df,attribute):
  Class=df.keys()[-1]
  target_variables=df[Class].unique()
  variables=df[attribute].unique()
  ent=0
  for variable in variables:
    entropy=0
    for target_variable in target_variables:
      num=len(df[attribute][df[attribute]==variable][df[Class]==target_variable])
      den=len(df[attribute][df[attribute]==variable])
      fraction=num/(den+eps)
      entropy+=-fraction*log(fraction+eps)
    fraction2=den/len(df)
    ent+=-fraction2*entropy
  return abs(ent)

def find_Info(df):
  Entropy_att=[]
  IG=[]
  for key in df.keys()[:-1]:
    IG.append(find_entropy(df)-conditional_entropy(df,key))
  return df.keys()[:-1][np.argmax(IG)]

def get_subtable(df,node,value):
  return df[df[node]==value].reset_index(drop=True)

def buildTree(df, used_attributes=None):
    Class = df.keys()[-1]
    node = find_Info(df)
    attValue = np.unique(df[node])

    if used_attributes is None:
        used_attributes = set()

    tree = {}
    tree[node] = {}

    if len(df.columns) - 1 == len(used_attributes):
        clValue, counts = np.unique(df[Class], return_counts=True)
        tree[node] = clValue[np.argmax(counts)]
        return tree

    for value in attValue:
        if value not in used_attributes:
            subtable = get_subtable(df, node, value)
            clValue, counts = np.unique(subtable[Class], return_counts=True)

            if len(counts) == 1:
                tree[node][value] = clValue[0]
            else:
                used_attributes.add(node)
                tree[node][value] = buildTree(subtable, used_attributes)

    return tree

tree=buildTree(Data)
pprint.pprint(tree)

{'Discount': {'No': {'Free Delivery': {'No': {'Holiday': {'No': 'No',
                                                          'yes': {'Holiday': 'No'}}},
                                       'Yes': {'Holiday': 'Yes'}}},
              'Yes': {'Holiday': 'Yes'}}}


C4.5

In [3]:
def find_entropy(df):
    class_column = df.columns[-1]
    entropy = 0
    class_counts = df[class_column].value_counts()
    total_instances = len(df)

    for count in class_counts:
        p = count / total_instances
        entropy -= p * log(p)

    return entropy

def conditional_entropy(df, attribute):
    class_column = df.columns[-1]
    target_variables = df[class_column].unique()
    variables = df[attribute].unique()
    entropy = 0

    for variable in variables:
        temp_entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute] == variable][df[class_column] == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + np.finfo(float).eps)
            temp_entropy -= fraction * log(fraction + np.finfo(float).eps)
        fraction2 = den / len(df)
        entropy += -fraction2 * temp_entropy

    return abs(entropy)

def find_info_gain(df):
    entropy_att = []
    info_gain = []
    split_info = []

    for key in df.keys()[:-1]:
        info_gain.append(find_entropy(df) - conditional_entropy(df, key))

        values_count = len(df[key].value_counts())
        split_info.append(-sum((x/len(df)) * log(x/len(df)) for x in range(1, values_count + 1)))

    gain_ratio = np.divide(info_gain, split_info)

    return df.keys()[:-1][np.argmax(gain_ratio)]

def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def build_tree(df, used_attributes=None):
    class_column = df.keys()[-1]
    node = find_info_gain(df)
    att_values = np.unique(df[node])

    if used_attributes is None:
        used_attributes = set()

    tree = {}
    tree[node] = {}

    if len(df.columns) - 1 == len(used_attributes):
        cl_value, counts = np.unique(df[class_column], return_counts=True)
        tree[node] = cl_value[np.argmax(counts)]
        return tree

    for value in att_values:
        if value not in used_attributes:
            subtable = get_subtable(df, node, value)
            cl_value, counts = np.unique(subtable[class_column], return_counts=True)

            if len(counts) == 1:
                tree[node][value] = cl_value[0]
            else:
                used_attributes.add(node)
                tree[node][value] = build_tree(subtable, used_attributes)

    return tree

tree = build_tree(Data)
pprint.pprint(tree)

{'Discount': {'No': {'Free Delivery': {'No': {'Holiday': {'No': 'No',
                                                          'yes': {'Holiday': 'No'}}},
                                       'Yes': {'Holiday': 'Yes'}}},
              'Yes': {'Holiday': 'Yes'}}}


Cart

In [5]:
def find_gini_index(df):
    Class = df.keys()[-1]
    gini_index = 1
    values = df[Class].unique()
    for value in values:
        p = len(df[df[Class] == value]) / len(df)
        gini_index -= p**2
    return gini_index

def conditional_gini_index(df, attribute):
    Class = df.keys()[-1]
    target_variables = df[Class].unique()
    variables = df[attribute].unique()
    gini_index = 0
    for variable in variables:
        weighted_gini = 0
        for target_variable in target_variables:
            num = len(df[attribute][(df[attribute] == variable) & (df[Class] == target_variable)])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + 1e-10)
            weighted_gini += (fraction**2)
        weighted_gini = 1 - weighted_gini
        fraction2 = den / len(df)
        gini_index += fraction2 * weighted_gini
    return abs(gini_index)

def find_split_attribute(df):
    min_gini_index = float('inf')
    split_attribute = None
    for attribute in df.keys()[:-1]:
        gini_index = conditional_gini_index(df, attribute)
        if gini_index < min_gini_index:
            min_gini_index = gini_index
            split_attribute = attribute
    return split_attribute

def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def buildTree(df, used_attributes=None):
    Class = df.keys()[-1]
    if used_attributes is None:
        used_attributes = set()

    node = find_split_attribute(df)
    att_values = np.unique(df[node])
    tree = {node: {}}

    if len(df.columns) - 1 == len(used_attributes):
        cl_value, counts = np.unique(df[Class], return_counts=True)
        tree[node] = cl_value[np.argmax(counts)]
        return tree

    for value in att_values:
        if value not in used_attributes:
            subtable = get_subtable(df, node, value)
            cl_value, counts = np.unique(subtable[Class], return_counts=True)

            if len(counts) == 1:
                tree[node][value] = cl_value[0]
            else:
                used_attributes.add(node)
                tree[node][value] = buildTree(subtable, used_attributes.copy())

    return tree

tree = build_tree(Data)
pprint.pprint(tree)

{'Discount': {'No': {'Free Delivery': {'No': {'Holiday': {'No': 'No',
                                                          'yes': {'Holiday': 'No'}}},
                                       'Yes': {'Holiday': 'Yes'}}},
              'Yes': {'Holiday': 'Yes'}}}
